In [ ]:
#################################################################################
#Loading Data
# https://docs.enthought.com/mayavi/mayavi/auto/examples.html

In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# #Importing Model Data
    
# dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# netCDF=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_test7tundra-7_062217.nc') #***
# true_time=netCDF['time']
# # parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_test5tundra-7_062217.nc') #***
# times=netCDF['time'].values/(1e9 * 60); times=times.astype(float);
# Np_str='125e3'
# #Restricts the timesteps of the data from timesteps0 to 140
# data=netCDF.isel(time=np.arange(0,140+1))
# # parcel=parcel.isel(time=np.arange(0,140+1))
# res='1km'

dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
netCDF=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_1e6.nc') #***
true_time=netCDF['time']
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_1e6.nc') #***
times=netCDF['time'].values/(1e9 * 60); times=times.astype(float);
Np_str='1e6'
#Restricts the timesteps of the data from timesteps0 to 140
data=netCDF
# data=netCDF.isel(time=np.arange(0,140+1))
# parcel=parcel.isel(time=np.arange(0,140+1))
res='1km'

# #uncomment if using 250m data
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# # # parcel=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***

# # Restricts the timesteps of the data from timesteps0 to 140
# data=data.isel(time=np.arange(0,400+1))
# # # parcel=parcel.isel(time=np.arange(0,400+1))
# res='250m'

In [ ]:
#################################################################################
#1km

In [ ]:
#Making binary array for qc+qi>1e-5 
fraction=2/8
def cloud_array(t):
    qc_t=data['qc'].isel(time=t)
    qi_t=data['qi'].isel(time=t)
    qcqi_t=qc_t+qi_t
    
    binary_t=np.zeros_like(qc_t)
    where=np.where(qcqi_t>=1e-6)
    binary_t[where]=1

    t_data=binary_t
    # t_data=binary_t[:,:,int(512*fraction):] 
    # t_data=t_data[:,:,0:150]
    # t_data=binary_t[:,:,:] #uncomment for full domain
    
    return t_data
t_data=cloud_array(100)
# plt.contourf(t_data[15,:,:]) 


#Making binary array for w>2
def updraft_array(t):
    w=data['winterp'].isel(time=t)
    
    binary_t=np.zeros_like(w)
    where=np.where(w>=3)
    binary_t[where]=1

    t_data=binary_t
    # t_data=binary_t[:,:,int(512*fraction):] 
    # t_data=t_data[:,:,0:150]
    # t_data=binary_t[:,:,:] #uncomment for full domain
    
    return t_data
w_t_data=updraft_array(100)


In [ ]:
# Reading Back Data Later
##############
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Binary_Array/'
in_file=dir2+f'lagrangian_binary_array_{res}_{Np_str}.h5'
with h5py.File(in_file, 'r') as f:
    # Load the dataset by its name
    Z_lagrangian = f['Z'][:]
    Y_lagrangian = f['Y'][:]
    X_lagrangian = f['X'][:]

    W = f['W'][:]

In [ ]:
#Final Plotting 
def single_plot(t,ax=None):
    import numpy as np
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    cmap='Blues'

    ########################################################################
    #get data for a single timestep
    t_data=cloud_array(t)
    
    #Preliminaries
    Nz = t_data.shape[0]
    Ny = t_data.shape[1]
    Nx = t_data.shape[2]

    if ax==None:
        fig = plt.figure(figsize=(12, 15))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_box_aspect([2, 1, 1])  # Aspect ratio (x, y, z)


    #ZZZZZZZZZ
    # Create a meshgrid for the x and y axes
    x = np.arange(t_data[0].shape[1])  # Use shape from the first slice for x dimension
    y = np.arange(t_data[0].shape[0])  # Use shape from the first slice for y dimension
    X, Y = np.meshgrid(x, y)
        
    # Iterate through the z-levels and plot a contour at each 
    for zind in np.arange(Nz):
        slice_z = t_data[zind]  # Extract the current z slice
        
        # Create a mask for points where data is 1
        mask = slice_z == 1  
        if np.any(mask):  # Check if there's any 1s in the current slice
            # Set the masked values to NaN to avoid plotting them
            z_masked = np.where(mask, slice_z, np.nan)
            
            # Create filled contours only where data is 1
            ax.contourf(X, Y, z_masked, offset=zind, alpha=0.5, cmap=cmap) #0.7


    #YYYYYYYYY
    # Create a meshgrid for the x and z axes (for the x–z plane)
    x = np.arange(Nx)
    z = np.arange(Nz)
    X,Z = np.meshgrid(x, z)
    
    # Iterate through the y-dimension and plot a contour for each y-level
    for yind in np.arange(Ny):
        # Extract the slice for this y (all z and x)
        y = t_data[:, yind, :]  # shape (Nz, Nx)
    
        # Create a mask for points where data equals 1
        mask = y == 1
        if np.any(mask):
            # Here we create a data array for the contour.
            # For demonstration, we fill with yind where mask is True, NaN otherwise.
            y_masked = np.where(mask, y, np.nan)
    
            # Plot the contour on the x–z plane at y = yind
            ax.contourf(X, y_masked, Z, zdir='y', offset=yind, alpha=0.5, cmap=cmap)

    #XXXXXXXXX           
    # Create a meshgrid for the y and z axes
    y = np.arange(t_data.shape[1])  # Use the second dimension (y)
    z = np.arange(t_data.shape[0])  # Use the first dimension (z)
    Y, Z = np.meshgrid(y, z)  # Create meshgrid for plotting
    
    # Iterate through the y-dimension and plot a contour at each y-level
    for xind in np.arange(Nx):  # Iterate through the y-dimension
        slice_x = t_data[:, :, xind]  # Extract the data slice for this y (all z levels for this y)
    
        # Create a mask for points where data is 1
        mask = slice_x == 1
        if np.any(mask):  # Check if there are any 1s in the current slice
            # Set the masked values to NaN to avoid plotting them
            x_masked = np.where(mask, slice_x, np.nan)  # Use yind values as the contour heights
    
            # Create filled contours only where data is 1 in the zx plane
            ax.contourf(x_masked, Y, Z, zdir='x', offset=xind, alpha=0.5, cmap=cmap)  
    


    
    ########################################################################
    #get data for a single timestep
    t_data=updraft_array(t)
    
    #Preliminaries
    Nz = len(data['zh'])  
    # Create a meshgrid for the x and y axes
    x = np.arange(t_data[0].shape[1])  # Use shape from the first slice for x dimension
    y = np.arange(t_data[0].shape[0])  # Use shape from the first slice for y dimension
    X, Y = np.meshgrid(x, y)
        
    # Iterate through the z-levels and plot a contour at each 
    for zind in np.arange(Nz):
        z = t_data[zind]  # Extract the current z slice
        ax.contour(X, Y, z, zdir='z', offset=zind, alpha=0.5,linewidths=0.2,colors='black')
    ########################################################################

   



    #Set Labels and Set z limit
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')
    ax.set_zlim(0, Nz)
    
    # Set the viewing angle, remove grid
    # ax.view_init(elev=30, azim=120)
    # ax.view_init(elev=10, azim=150)
    ax.view_init(elev=10, azim=130)
    ax.dist = 10
    # ax.set_facecolor('blue')
    ax.grid(False)
    
    # Remove the back of the axis
    ax.xaxis.pane.fill = True
    ax.yaxis.pane.fill = True
    ax.zaxis.pane.fill = False
    ax.xaxis.pane.set_facecolor('blue')   # Set x-pane to blue
    ax.yaxis.pane.set_facecolor('blue')   # Set y-pane to blue
    ax.zaxis.pane.set_facecolor('green')   # Set y-pane to blue
    # ax.xaxis.pane.set_facecolor((0.0, 0.0, 0.0, 0.0))  # Make back pane transparent
    # ax.yaxis.pane.set_facecolor((0.0, 0.0, 0.0, 0.0))
    # ax.zaxis.pane.set_facecolor((0.0, 0.0, 0.0, 0.0))

    #LAND SURFACE
    #shade in the bottom green for land
    # # Clear previous scatter points
    # if hasattr(single_plot, 'plane_obj'):
    #     single_plot.plane_obj.remove()  # Remove previous scatter plot if it exists
        
    x_plane = np.linspace(Nx*fraction, Nx+1, 50) 
    y_plane = np.linspace(0, Ny+20, 50) 
    x_plane, y_plane = np.meshgrid(x_plane, y_plane)
    z_plane = np.zeros_like(x_plane)  # Set Z = 0 for the plane
    single_plot.plane_obj =ax.plot_surface(x_plane, y_plane, z_plane, color='green', alpha=0.5,label='land',antialiased=True)
    
    # #FIX X,Y LIMS
    ax.set_xlim((0,Nx))
    ax.set_ylim((0,Ny))
    ax.set_zlim((0,Nz))

    #plotting lagrangian parcels
    max_p=5000
    where_updraft=np.where(W[t,:]>=0.1)
    zs=Z_lagrangian[t,where_updraft][0][0:max_p]
    ys=Y_lagrangian[t,where_updraft][0][0:max_p]
    xs=X_lagrangian[t,where_updraft][0][0:max_p]

    # # Clear previous scatter points
    # if hasattr(single_plot, 'scatter_obj'):
    #     single_plot.scatter_obj.remove()  # Remove previous scatter plot if it exists

    # Plot the new scatter points
    single_plot.scatter_obj = ax.scatter(xs, ys, zs, color='red', s=0.1)

single_plot(t=67)

In [ ]:
#MAKING ANIMATION

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

# Function to generate the animation
def create_animation(start_t, end_t, output_file, fps=2):
    fig = plt.figure(figsize=(12, 15))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_box_aspect([2, 1, 1])  # Aspect ratio (x, y, z)

    # Define the update function for the animation
    def update(t):
        if np.mod(t,15)==0: print(f'current t: {t}')
        ax.clear()
        single_plot(t,ax)

    # Create the animation object
    ani = FuncAnimation(fig, update, frames=np.arange(start_t, end_t + 1), repeat=False)

    # Save the animation as a GIF file using PillowWriter
    writer = PillowWriter(fps=fps)
    ani.save(output_file, writer=writer)

# Usage
output_filename = dir+'Animations/Cloud_Animations/'+f'output_animation_{res}.gif'
# create_animation(start_t=0, end_t=len(data['time'])-1, output_file=output_filename)
create_animation(start_t=60, end_t=63, output_file=output_filename) #TESTING

In [ ]:
#CONVERTING TO MP4
def convert_gif_to_mp4(input_file, output_file, fps,speed,bitrate='750k'):
    from moviepy.editor import VideoFileClip, vfx
    # Load the GIF file
    gif_clip = VideoFileClip(input_file)

    # Set the desired framerate if provided
    if fps:
        gif_clip = gif_clip.set_fps(fps)
    if speed != 1.0:
        gif_clip = gif_clip.fx(vfx.speedx, speed)

    # Write the GIF as an MP4 file
    gif_clip.write_videofile(output_file, codec="libx264",bitrate=bitrate)



#RUNNING CONVERSION

input_filename = dir+'Animations/Cloud_Animations/'+f'output_animation_{res}.gif'
output_filename = dir+'Animations/Cloud_Animations/'+f'output_animation_{res}.mp4'
if res=='1km':
    convert_gif_to_mp4(input_filename, output_filename, fps=None,speed=2,bitrate='750k')  # Optional fps argument
elif res=='250m':
    convert_gif_to_mp4(input_filename, output_filename, fps=None,speed=10,bitrate='750k')  # Optional fps argument